# Peforth magics %ai and %chat

In [ ]:
_file_ = "Jupyter Notebooks/OpenAI/ChatLKK/develop/peforth AI magics.ipynb"

## 祭出 peforth 
peforth 有 %f, %ai 以及 %chat 三個 Jupyter notebook 的 magics.

|Command|Description|
|---|:---|
|**%f**|FORTH statement, 例如： ." Hello World!", help 等等。任意組合 commands, 管理大量 commands 的交談式語言，非常精簡、優美。|
|**%ai**|對 AI 發問，不帶歷史。例如： Tell me a joke.|
|**%chat**|對 AI 發問，帶上歷史。例如： Why is it funny ?|


In [ ]:
import os, peforth

In [ ]:
from columbus_api import Columbus
columbus = Columbus()

## 準備工作，為 AI 提供必要零件。
從 columbus_api 取得 llm. 這個例子是取用 LLM for LangChain, 如果是別的，例如 LLM for CrewAI, LLM for DSPy 則以下的 llm_wrapper 會稍有不同。

### 採用 OpenAI 

In [ ]:
# 查看 api key 採用 columbus if is null, otherwise 用你自己的 account.
os.getenv("OPENAI_API_KEY")

In [ ]:
llm = columbus.get_llm_for_LangChain(modelname="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))
%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_object \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_object :> invoke(pop()).content ; \ 定義 llm_wrapper
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

### 採用 Gemini

In [ ]:
# 確定有 api key 
os.getenv("GoogleAIStudio_API_KEY")

In [ ]:
llm = columbus.get_llm_gemini(api_key=os.getenv("GoogleAIStudio_API_KEY"))
%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_function \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_function :> (pop()) dup str :> rfind("candidates")==-1 if str else :> ['candidates'][0]['output'] then ;
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

### 採用 Antropic Claude

- [API reverence - SDK](https://docs.anthropic.com/en/api/client-sdks)
- [Models the prices](https://docs.anthropic.com/en/docs/about-claude/models#legacy-model-comparison)

In [ ]:
# 確定有 api key 
os.getenv("ANTHROPIC_API_KEY")

In [ ]:
!pip install anthropic
import anthropic
client = anthropic.Anthropic()
def llm(prompt):
    message = client.messages.create(
        model="claude-3-haiku-20240307", # $0.25 / $1.25 MTok
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return message

%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_function \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_function :> (pop()) dup str :> rfind("type='message'")==-1 if str else :> content[0].text then ;
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

## 試用 magics %ai and %chat 

In [ ]:
%f "" to chat_history \ 把以前的 chatting contexts 都清除掉。
%ai tell me a joke 
%chat why is it funny ? 

## 查看 chat_history

In [ ]:
%f chat_history . \ 最後有個小點，它是 forth 的 print 命令。

## 篡改 chat_history

In [ ]:
%%f
<text>
User: Write me a poem.
Assistnat: What peom?
User: peforth new magic %ai and %chat for Jupyter notebook is now available.
</text> to chat_history

In [ ]:
%chat

# Playground

In [ ]:
%ai Tell me about 愛沙尼亞
%chat Their communist experience.  

# The END